In [1]:
import smartpy as sp

SyntaxError: invalid syntax (2762607428.py, line 30)

# Contract

class ClassSchedule(sp.Contract):
    def __init__(self, teacher_address,student_address):
        self.init(teacher_address = teacher_address,
                  student_address=student_address,
                  teacherchoices = sp.map(tkey = sp.TString),
                  studentchoice = sp.map(tkey = sp.TString),
                  student_balance=0,
                  teacher_balance=0
                  )
    @sp.entry_point
    def teacherchoice(self, params):
        self.data.teacherchoices[params.time] = sp.record(day = params.day,
                                                          type = params.type)
    @sp.entry_point
    def studentchoice(self,params):
        self.data.studentchoice[params.time] = sp.record(day = params.day)
    
    @sp.entry_point
    def student_balance(self,params):
        self.data.student_balance=params.op1

    @sp.entry_point
    def teacher_balance( self,params):
        self.data.teacher_balance=params.op2

    @sp.entry_point
    def transfer(self,params):
        sp.if  (self.data.teacherchoices.contains(params.time)):
            sp.if (self.data.studentchoice.contains(params.time)):
                sp.if self.data.student_balance >0:
                    sp.verify(sp.sender == self.data.student_address)
                    self.data.teacher_balance+= 5
                    self.data.student_balance-= 5

# Adding Scenarios

In [ ]:
@sp.add_test(name = "First_test")
def test():
    teacher_address = sp.address("tz1-teacher-address-1234")
    student_address = sp.address("tz1-student-address-5678")
    
    scenario = sp.test_scenario()
    contract = ClassSchedule(teacher_address,student_address)
    scenario += contract

    scenario +=contract.teacher_balance(op2=10).run(sender = sp.address("tz1-student-address-5678"))
    scenario +=contract.student_balance(op1=5).run(sender = sp.address("tz1-student-address-5678"))


    scenario +=contract.teacherchoice(
                                  time="10:00",
                                  day='11-05-22',
                                  type="beginner").run(sender = sp.address("tz1-teacher-address-1234"))

    scenario +=contract.studentchoice(
                                  time="10:00",
                                  day='11-05-22').run(sender = sp.address("tz1-student-address-5678"))             
       
    scenario +=contract.transfer(time="10:00",
                                  day='11-05-22',
                                  type="beginner").run(sender = sp.address("tz1-student-address-5678"))